# Traitements des images

In [1]:
import os
from PIL import Image

# Augmenter la taille max des images
Image.MAX_IMAGE_PIXELS = 100000000

Les images tests (résolution 0.1) sont contenues dans `/data/download_test/` et sont au format `EEEE-NNNN.tif`.

Les images font 10'000 x 10'000 pixels, il faut donc augmenter la limite de PIL pour que les images puissent être lues.

In [2]:
def formatter_nombre(nombre):
    # Convertir le nombre en chaîne
    nombre_str = str(nombre)
    
    # S'il est décimal
    if '.' in nombre_str:
        # Si décimal entier, le convertir en int
        if nombre.is_integer():
            return str(int(nombre))
        # Sinon, retourner sans décimale
        else:
            return nombre_str.replace('.', '')
    # Si le nombre est un entier, le retourner tel quel
    else:
        return nombre_str

In [3]:
# input_dir = "../data/download_test/"
input_dir = "../data/download_by_bbox/"
output_dir = "../data/img_reduites/"

À partir d'une taille d'image, découper les originales en petits carrés, dont le nom correspond aux coordonnées

In [4]:
# Définir les tailles d'image à tester
# image_sizes = [500, 1000, 2000]
image_sizes = [1000]

# Pour chaque taille choisie
for size in image_sizes:
    # Définir les nouvelles largeur et hauteur
    new_width, new_height = size, size

    # Doit diviser 10'000 en entier
    if 10000 % new_width != 0:
        raise Exception("Doit être un diviseur de 10'000")

    # Correspond au nombre de division effectuées
    divisions = 10000 // new_width

    # Ajuster le dir en fonction
    output_dir_size = output_dir + str(size) + "/"

    # Pour chaque image de base
    for filename in os.listdir(input_dir):

        #  Ouvrir l'image
        img = Image.open(os.path.join(input_dir, filename))

        # Récupérer les coordonnées est (x) et nord (y)
        coords = filename.split(".")[0]
        ecoord, ncoord = coords.split('-')
        
        # Itérer pour chaque membre de la mosaïque
        for e in range(divisions):
            for n in range(divisions):
                left = e * new_width
                upper = n * new_height
                right = (e + 1) * new_width
                lower = (n + 1) * new_height

                # Croper l'image avec les nouvelles coordonnées
                new_image = img.crop((left, upper, right, lower))

                # Calculer l'échelle de changement
                coords_augm = new_width / 1000
                
                # Calculer les coordonnées hectométriques
                ecoord_compl = formatter_nombre(e*coords_augm)
                # PIL part du haut gauche alors que les coords de l'image sont en bas à gauche
                ncoord_compl = formatter_nombre((divisions-n-1)*coords_augm)

                # Enregistrer l'image avec les nouvelles coordonnées
                new_filename = f"{ecoord}{ecoord_compl}-{ncoord}{ncoord_compl}.tif"
                new_image.save(os.path.join(output_dir_size, new_filename))

Convertir les images du dossier de .tif à .jpg

In [13]:
img_input_dir = output_dir + '/1000'
img_output_dir = img_input_dir + '/jpg'

for filename in os.listdir(img_input_dir):
    with Image.open(os.path.join(img_input_dir, filename)) as img:
        # Créer le nom et chemin
        out_name = os.path.splitext(filename)[0] + '.jpg'
        out_path = os.path.join(img_output_dir, out_name)
        
        # Exporter en jpeg
        img.save(out_path, 'JPEG')